## Setup

Recomendações de Hiperparâmetros para o Ajuste Fino (Fine-tuning) do BERT
Recomendações dos autores do BERT para a escolha dos principais hiperparâmetros durante o processo de ajuste fino, conforme descrito no artigo original:

Tamanho do Lote (Batch size): 16 ou 32.

Taxa de Aprendizagem (Learning rate): Para o otimizador AdamW, valores ideais geralmente estão na faixa:

2e-5, 3e-5, 5e-5

Número de Épocas (Epochs): O treinamento é tipicamente executado por 2, 3 ou 4 épocas, pois um número maior pode levar a um overfitting rapidamente.

É crucial notar a relação inversa entre o tamanho do lote e a precisão do modelo. Embora lotes maiores possam acelerar o tempo de treinamento (utilizando mais a capacidade da GPU), eles podem levar a uma precisão ligeiramente inferior, pois o modelo tem menos oportunidades de atualizar seus pesos com base em diferentes subconjuntos de dados. Para otimizar a generalização, lotes menores (como 16) são frequentemente preferidos.

In [ ]:
# Instalar PyTorch, TorchVision e Torchaudio
%pip install torch torchvision torchaudio

# Instalar outras bibliotecas necessárias
%pip install transformers==4.29.2 numpy pandas seaborn matplotlib scikit-learn

# Instalar o watermark para exibir versões das bibliotecas
%pip install -U watermark

In [ ]:
%reload_ext watermark

# Exibir versões das bibliotecas
%watermark -v -p numpy,pandas,torch,transformers

In [ ]:
!nvidia-smi

In [ ]:
# A partir da biblioteca Transformers da Hugging Face, importando as classes
import transformers
from transformers import AutoModel # ou AutoModel, para carregar o modelo BERT
from transformers import AutoTokenizer  # ou AutoTokenizer, para carregar o tokenizador BERT
from transformers import get_linear_schedule_with_warmup  # Para controle do agendamento da taxa de aprendizado

# Importando a biblioteca PyTorch
import torch
from torch.optim import AdamW # Para otimização do modelo
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from collections import defaultdict
from textwrap import wrap

import os
# from google.colab import files  # Específico do Google Colab

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='viridis', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Modelos
BERTimbauBase = 'neuralmind/bert-base-portuguese-cased'
BERTimbauLarge = 'neuralmind/bert-large-portuguese-cased'
BERMultilingualBase = 'google-bert/bert-base-multilingual-cased' # Modelo BERT multilíngue do Google
XLMRobertaBase = 'FacebookAI/xlm-roberta-base' # Modelo XLM-RoBERTa é uma versão multilíngue do RoBERTa.

PRE_TRAINED_MODEL_NAME = BERTimbauBase

# Carregando o modelo BERTimbau da Hugging Face pré-treinado em português
base_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False)

# Carregando o tokenizador BERTimbau da Hugging Face pré-treinado em português
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Configuração de treino
EPOCHS = 10
# Teste e validação terão cada um a metade do TEST_SIZE
TEST_SIZE = 0.3
# Taxa de dropout para regularização, ajudando a prevenir o sobreajuste (overfitting).
DROPOUT = 0.3
# O parâmetro WEIGHT_DECAY=0.01 ajuda a reduzir overfitting penalizando pesos muito grandes.
WEIGHT_DECAY = 0.01

# Tamanho do lote "BATCH" (Quantos exemplos um conjunto de dados vai ter) # 4, 8, 16
BATCH_SIZE = 16
# Taxa de aprendizado (Learning Rate) para o otimizador # 2e-5, 3e-5 ou 5e-5
LEARNING_RATE = 3e-5
# Habilita ou desabilita o uso de pesos para balanceamento de classes
CLASS_WEIGHTS = True

dataset_name = 'reviews_3000_limpeza_demojize.csv'
score_column = 'score'
sentiment_column = 'sentiment'

class_names = [
        'extremamente negativo',
        'negativo',
        'neutro',
        'positivo',
        'extremamente positivo',
    ]


In [ ]:
print(f"Classe do modelo carregado: {type(base_model)}")
print(f"Classe do tokenizer carregado: {type(tokenizer)}")

In [ ]:
# Verificando se a GPU com suporte a CUDA está disponível no sistema
torch.cuda.is_available()

In [ ]:
device

## Manipulação dos dados

## Importação dos dados

In [ ]:
# uploaded = files.upload()

In [ ]:
df = pd.read_csv(dataset_name)


def get_sentiment(rating):
    rating = int(rating)
    if rating == 1:
        return 0
    elif rating == 2:
        return 1
    elif rating == 3:
        return 2
    elif rating == 4:
        return 3
    else:
        return 4


df[sentiment_column] = df[score_column].apply(get_sentiment)

df.head(2)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.sentiment.value_counts()

In [ ]:
# Contar as avaliações por score
sentiment_counts = df.sentiment.value_counts()

# Plotar as contagens por sentimento
sns.barplot(x=class_names, y=sentiment_counts.values, palette="viridis")

plt.xlabel("Sentimento")
plt.ylabel("Número de Avaliações")

## Data Processing

Antes de alimentar textos para o modelo BERT, é necessário realizar algumas etapas de pré-processamento. O BERT requer um formato específico de entrada, e uma das etapas essenciais é a criação de vetores de 0s e 1s chamados attention mask, que indicam quais tokens devem ser considerados válidos, e a adição de três tokens especiais aos textos:

* [SEP] (102)- Marca o fim de uma frase
* [CLS] (101)- Deve ser colocado no inicio de cada frase para o BERT saber que trata-se de um problema de classificação.
* [PAD] (0)- Tokens de valor 0 que devem ser adicionados às sentenças para garantir que todas tenham o mesmo tamanho.

Esses tokens podem ser adicionados utilizando o método [encode_plus](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode_plus) da Hugging Face. Esse método realiza várias operações essenciais, como:

* Tokenizar o texto: Converte as palavras em IDs de tokens que correspondem ao vocabulário do modelo BERT.
Adicionar tokens especiais: Insere automaticamente os tokens [CLS], [SEP] e [PAD] quando necessário.
* Realizar padding (preenchimento): Preenche as sentenças mais curtas com tokens [PAD] até atingir o comprimento máximo especificado.
* Gerar máscara de atenção (attention mask): Cria uma máscara que indica quais tokens são reais (com valor 1) e quais são de preenchimento (com valor 0), ajudando o modelo a focar nos tokens importantes.
* Controle de truncamento: Garante que textos mais longos sejam truncados para o comprimento máximo permitido.
* Retornar tensores: Converte os resultados para tensores PyTorch ou TensorFlow, prontos para uso em modelos de aprendizado de máquina.

1. **Tokens Especiais:**
* [CLS] (101): Deve ser colocado no início de cada frase para que o BERT saiba que trata-se de um problema de classificação.
* [SEP] (102): Marca o fim de uma frase ou sequência.
* [PAD] (0): Tokens de valor 0 que devem ser adicionados às sentenças para garantir que todas tenham o mesmo tamanho (padding).
[UNK] (100): Token utilizado para palavras desconhecidas, que não foram vistas durante o treinamento. Ex:
 * 101 -> [CLS]
 * 2146 -> "O"
 * 1004 -> "BERT"
 * 2003 -> "é"
 * 1037 -> "uma"
 * 4600 -> "ferramenta"
 * 2395 -> "poderosa"
 * 102 -> [SEP]

2. **Padding (Preenchimento):**
* O BERT espera que todas as entradas tenham o mesmo comprimento. Isso significa que você precisará preencher (pad) as sentenças mais curtas com tokens de preenchimento (como 0) até um comprimento fixo. Isso é necessário para processamento em lotes (batches) eficiente. Ex:
 * 101 -> [CLS]
 * 2146 -> "O"
 * 1004 -> "BERT"
 * 2003 -> "é"
 * 1037 -> "uma"
 * 4600 -> "ferramenta"
 * 2395 -> "poderosa"
 * 102 -> [SEP]
 * 0 -> [PAD]
 * 0 -> [PAD]
 * 0 -> [PAD]
 * 0 -> [PAD]

3. **Máscara de Atenção (Attention Mask):**
* A máscara de atenção é uma matriz que indica quais tokens são reais (com valor 1) e quais são de preenchimento (com valor 0). Isso ajuda o BERT a se concentrar nos tokens importantes e ignorar os preenchimentos. Ex:
 * 101 -> 1
 * 2146 -> 1
 * 1004 -> 1
 * 2003 -> 1
 * 1037 -> 1
 * 4600 -> 1
 * 2395 -> 1
 * 102 -> 1
 * 0 -> 0
 * 0 -> 0
 * 0 -> 0
 * 0 -> 0

#### Exemplo do processo de tokenização

In [ ]:
sample_txt = "Santa Helena é uma cidade"

tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f" Sentence: {sample_txt}")
print(f"   Tokens: {tokens}")
print(f"Token IDs: {token_ids}")

In [ ]:
# [SEP] - Marcador para indicar o fim de uma sentença.
tokenizer.sep_token, tokenizer.sep_token_id

In [ ]:
# [CLS] - Token que deve ser adicionado no início de cada sentença.
tokenizer.cls_token, tokenizer.cls_token_id

In [ ]:
# [PAD] - Token de preenchimento utilizado para garantir que todas as sentenças tenham o mesmo tamanho (padding).
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
# [UNK] - Token de palavras desconhecidas, usadas para palavras que não apareceram no conjunto de treinamento do modelo.
tokenizer.unk_token, tokenizer.unk_token_id

Todo esse trabalho pode ser feito usando o método [encode_plus](https://huggingface.co/transformers/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.encode_plus):

In [ ]:
# Usa o método encode_plus() para preparar os dados para o modelo
encoding = tokenizer.encode_plus(
    sample_txt,  # Texto a ser tokenizado
    add_special_tokens=True,  # Adiciona os tokens especiais [CLS] no início e [SEP]
    max_length=9,  # Define o comprimento máximo da sequência "entrada" (160 tokens "palavras" neste caso)
    truncation=True,  # Trunca a sequência se ultrapassar o comprimento máximo
    padding="max_length",  # Realiza o padding, adicionando tokens de preenchimento [PAD] até o comprimento máximo
    return_token_type_ids=False,  # Não retorna os 'token_type_ids', pois não são necessários para analise de sentimentos
    return_attention_mask=True,  # Retorna a máscara de atenção, indicando quais tokens devem ser considerados (1 para tokens reais, 0 para padding)
    return_tensors="pt",  # Retorna os resultados em formato de tensor PyTorch
)

# O dicionário resultante contém 'input_ids' (sequência de tokens) e 'attention_mask' (máscara de atenção)
encoding.keys()

In [ ]:
encoding

In [ ]:
# Os IDs dos tokens agora são armazenados em um Tensor e preenchidos até um comprimento de 160 colunas
print(
    len(encoding["input_ids"][0])
)  # Exibe o comprimento da sequência de tokens (esperado 160, incluindo tokens especiais e padding)
# Exibe o tensor contendo os IDs dos tokens, incluindo o [CLS], [SEP] e tokens de preenchimento [PAD]
encoding["input_ids"]

# A máscara de atenção tem o mesmo comprimento:
print(len(encoding["attention_mask"][0]))
# Exibe a máscara de atenção (1 para tokens reais, 0 para tokens de padding)
encoding["attention_mask"]

In [ ]:
# Converte os IDs dos tokens de volta para tokens
tokenizer.convert_ids_to_tokens(encoding["input_ids"][0])

##### Exemplo: Codificação do nosso texto de exemplo:

In [ ]:
# Passa os tokens de entrada e a máscara de atenção para o modelo BERT
last_hidden_state, pooled_output = base_model(
    input_ids=encoding[
        "input_ids"
    ],  # IDs dos tokens, gerados pelo tokenizer a partir do texto de entrada
    attention_mask=encoding["attention_mask"],  # Máscara de atenção
)

# last_hidden_state é a saída da última camada oculta do modelo BERT
# Ele contém as representações de cada token no contexto da frase inteira
# Forma: [batch_size, sequence_length, hidden_size]
# Ex: Para um modelo BERT Base, a dimensão será [batch_size, 768]

# pooled_output é a saída correspondente ao token [CLS] (usado para tarefas de classificação)
# É como um resumo da frase inteira, ou seja, é uma única representação para a frase
# Geralmente, usado para tarefas de classificação
# Forma: [batch_size, hidden_size]
# Ex: Para BERT Base, será [batch_size, 768]

In [ ]:
# Forma: [batch_size, sequence_length, hidden_size]
# Last_hidden_state é a saída da última camada oculta do modelo BERT, contendo as representações de cada token no contexto da frase inteira
last_hidden_state.shape

In [ ]:
# Forma: [batch_size, hidden_size]
# Lembrando que pooled_output é a saída correspondente ao token [CLS], que é uma representação da frase inteira, usada para classificação
pooled_output.shape

O `last_hidden_state` é uma sequência de estados ocultos da última camada do modelo. A obtenção do `pooled_output` é feita aplicando o [BertPooler](https://github.com/huggingface/transformers/blob/edf0582c0be87b60f94f41c659ea779876efc7be/src/transformers/modeling_bert.py#L426) em `last_hidden_state`.

`pooled_output` é a saída correspondente ao token [CLS] (usado para tarefas de classificação)
Podemos pensar como um resumo da frase inteira, ou seja, é uma única representação para a frase.

In [ ]:
# Obtém o tamanho do vetor de características ocultas (hidden_size) configurado no modelo BERT
# Para BERT Base, o valor será 768. Para BERT Large, será 1024.
base_model.config.hidden_size

#### Escolhendo o comprimento da sequência

BERTimbau Base e BERTimbau Large: as entradas podem ter até 512 palavras (tokens)

Para escolher o comprimento máximo de sequência, vamos armazenar o número de tokens de cada review

In [ ]:
token_lens = []

for txt in df.content:
    tokens = tokenizer.encode(txt, truncation=True, max_length=512)
    token_lens.append(len(tokens))

Exibir a distribuição

In [ ]:
sns.histplot(token_lens, bins=30, kde=True)  # kde=True adiciona a curva de densidade

# Define os rótulos e o título
plt.xlabel("Contagem de Tokens")
plt.ylabel("Frequência")
plt.title("Distribuição do Número de Tokens")

# Encontra e exibe o número máximo de tokens no gráfico
MAX_LEN = max(token_lens)
plt.axvline(MAX_LEN, color="r", linestyle="--", label=f"Máximo de Tokens: {MAX_LEN}")

# Adiciona uma legenda para explicar a linha vermelha
plt.legend()

# Salva a imagem em um arquivo
plt.savefig("distribuicao_tokens.png")

# Exibe o gráfico (opcional, dependendo do seu ambiente de execução)
plt.show()

In [ ]:
max_len_val = max(token_lens)

MAX_LEN = 512 if max_len_val >= 512 else max_len_val

print(f"O comprimento máximo de tokens é: {MAX_LEN}")

#### Dividindo o conjunto de dados:

In [ ]:
# Divide o DataFrame em conjunto de treinamento, teste e validação
# Primeiro divide em treino e teste
df_train, df_test = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_SEED)

# Divide o conjunto de teste em dois, criando um conjunto de validação (50% do teste) e o conjunto de teste final (50% do teste)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

df_train.shape  # Tamanho do conjunto de treinamento
df_val.shape  # Tamanho do conjunto de validação
df_test.shape  # Tamanho do conjunto de teste

#### Criando o Dataset

In [ ]:
# Estende a classe Dataset do PyTorch
class GPReviewDataset(Dataset):

    # Inicializa o dataset com os reviews, targets (sentimentos), tokenizador e o comprimento máximo da sequência
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews  # Armazena os textos das avaliações
        self.targets = targets  # Armazena os sentimentos das avaliações
        self.tokenizer = tokenizer  # Tokenizador BERT
        self.max_len = max_len  # Comprimento máximo da sequência

    # Método __len__ retorna o número de exemplos no dataset (Númeor de reviews)
    def __len__(self):
        return len(self.reviews)

    # Método __getitem__ recupera um item individual do dataset
    def __getitem__(self, item):
        review = str(self.reviews[item])  # Obtém o review atual e o converte em string
        target = self.targets[item]  # Obtém o rótulo correspondente ao review atual

        # Codifica o review em IDs de tokens e máscaras de atenção usando o tokenizador BERT
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,  # Adiciona tokens especiais como [CLS] e [SEP]
            max_length=self.max_len,  # Define o comprimento máximo para truncamento/preenchimento
            truncation=True,  # Trunca a sequência se ultrapassar o comprimento máximo
            return_token_type_ids=False,  # Não retorna os IDs de tipo de token, pois não é necessário
            padding="max_length",  # Preenche as sequências até o comprimento máximo
            return_attention_mask=True,  # Retorna a máscara de atenção que indica os tokens válidos
            return_tensors="pt",  # Retorna os tensores prontos para PyTorch
        )

        # Retorna um dicionário com o texto do review, os IDs dos tokens, a máscara de atenção e o rótulo
        return {
            "review_text": review,  # O review original em texto
            "input_ids": encoding[
                "input_ids"
            ].flatten(),  # IDs dos tokens achatados em uma dimensão
            "attention_mask": encoding[
                "attention_mask"
            ].flatten(),  # Máscara de atenção achatada
            "targets": torch.tensor(
                target, dtype=torch.long
            ),  # Rótulo convertido para tensor PyTorch de tipo long
        }

Criar alguns carregadores de dados. Aqui está uma função auxiliar para fazer isso:

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle: bool):
    # Cria um dataset GPReviewDataset a partir dos dados de entrada
    ds = GPReviewDataset(
        reviews=df.content.to_numpy(),  # Converte a coluna dos reviews (content) para um array numpy
        targets=df[
            sentiment_column
        ].to_numpy(),  # Converte a coluna de sentimentos para um array numpy
        tokenizer=tokenizer,  # Tokenizador do BERT para processar os reviews
        max_len=max_len,  # Comprimento máximo para o padding/truncamento
    )

    # Retorna um DataLoader para o dataset, que divide os dados em lotes
    return DataLoader(
        dataset=ds,  # O dataset criado
        batch_size=batch_size,  # Tamanho do lote
        # num_workers=4,           # Número de processos para usar para carregar os dados.
        num_workers=0,  # Evita o multiprocessamento, usando um único processo para carregar os dados
        shuffle=shuffle,  # Embaralha ou não dados a cada época para melhorar a generalização do modelo
    )

Criando DataLoaders para os conjuntos de dados de treinamento, validação e teste

In [ ]:
# Cada DataLoader vai conter os dados em lotes, facilitando o treinamento do modelo
# Dados de treinamento são embaralhados (shuffle = True)
train_data_loader = create_data_loader(
    df_train, tokenizer, MAX_LEN, BATCH_SIZE, shuffle=True
)
val_data_loader = create_data_loader(
    df_val, tokenizer, MAX_LEN, BATCH_SIZE, shuffle=False
)
test_data_loader = create_data_loader(
    df_test, tokenizer, MAX_LEN, BATCH_SIZE, shuffle=False
)

In [ ]:
print('Numero de lotes "batches" de treinamento:', len(train_data_loader))
print('Tamanho dos lotes "batches" de treinamento:', train_data_loader.batch_size)

print('Numero de lotes "batches" de validação:', len(val_data_loader))
print('Tamanho dos lotes "batches" de validação:', val_data_loader.batch_size)

print('Numero de lotes "batches" de teste:', len(test_data_loader))
print('Tamanho dos lotes "batches" de teste:', test_data_loader.batch_size)

## Criando o Modelo

In [ ]:
# Definindo o Classificador de Sentimento (BERT + Dropout + Camada Final) para Classificação.
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()

        # Inicializa o modelo BERT pré-treinado.
        # O BERT será responsável por gerar as representações contextuais dos textos de entrada.
        # O `return_dict=False` significa que o retorno será uma tupla e não um dicionário.
        self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False)

        # Camada de Dropout para regularização durante o treinamento.
        # Dropout é uma técnica que desativa aleatoriamente algumas conexões entre neurônios,
        # ajudando a evitar overfitting.
        self.drop = nn.Dropout(p=DROPOUT)

        # Camada final totalmente conectada (linear), que mapeia o vetor de saída do BERT para a quantidade de classes de saída.
        # O número de features de entrada (in_features) é o tamanho da saída do BERT (geralmente 768).
        # O número de classes (out_features) é especificado pelo parâmetro `n_classes`, que no caso é 5 para sentimentos.
        self.out = nn.Linear(
            in_features=self.bert.config.hidden_size, out_features=n_classes
        )

    def forward(self, input_ids, attention_mask):
        # O método forward define o que acontece quando o modelo recebe dados de entrada.
        # O modelo BERT gera uma tupla com duas saídas: sequence_output e pooled_output.
        # A variável 'pooled_output' é a saída do token [CLS], que será usada para classificação.
        # `input_ids`: ids dos tokens que representam o texto.
        # `attention_mask`: mascara que indica quais tokens devem ser atendidos (ignora padding).

        # `pooled_output` é a representação do token [CLS], que é utilizado para tarefas de classificação.
        _, pooled_output = self.bert(
            input_ids=input_ids,  # IDs dos tokens de entrada
            attention_mask=attention_mask,  # Máscara de atenção para ignorar tokens de padding
        )

        # Aplica o Dropout sobre o pooled_output para regularização.
        # Durante o treinamento, algumas conexões são "desligadas" para evitar overfitting.
        output = self.drop(pooled_output)

        # A saída do Dropout é passada pela camada totalmente conectada (linear) para gerar as previsões de classe.
        # Aqui, `self.out(output)` retorna os logits para cada classe (por exemplo, positivo, neutro, negativo).
        return self.out(output)

Usamos uma camada dropout para alguma regularização e uma camada totalmente conectada para nossa saída. Observe que estamos retornando a saída bruta da última camada, pois isso é necessário para que a função cross-entropy loss function no PyTorch funcione.

In [ ]:
# Instanciar o modelo de classificação de sentimentosmovê-lo para a GPU:
# O número de classes é 5 (extremamente negativo, negativo, neutro, positivo, extremamente positivo).
model = SentimentClassifier(len(class_names))
model = model.to(device)

### Exemplo

In [ ]:
# EXEMPLO
# Obtendo o primeiro lote de dados do train_data_loader
data = next(iter(train_data_loader))
# Exibindo as dimensões dos tensores de um lote de dados
print(
    "input_ids shape: ", data["input_ids"].shape
)  # Formato esperado: [batch_size, max_len] para os IDs dos tokens dos reviews
print(
    "attention_mask shape: ", data["attention_mask"].shape
)  # Formato esperado: [batch_size, max_len] para a máscara de atenção
print(
    "targets shape: ", data["targets"].shape
)  # Formato esperado: [batch_size] para os rótulos (sentimentos)

# Saida [batch_size, max_len]
# 'batch_size' refere-se ao número de amostras no lote.
# 'max_len'  refere-se ao número máximo de tokens por amostra (É o valor de MAX_LEN que foi configurado anteriormente).

In [ ]:
# EXEMPLO
# Move os 'input_ids' e 'attention_mask' para o dispositivo correto (GPU ou CPU)
# 'input_ids' contém os IDs dos tokens do texto de entrada, já processados pelo tokenizador do BERT
# 'attention_mask' contém uma máscara que indica quais tokens são válidos (1) e quais são padding (0)
input_ids = data["input_ids"].to(device)
attention_mask = data["attention_mask"].to(device)

## model(input_ids, attention_mask):
# Esta parte passa os dados de entrada para o modelo (input_ids e attention_mask)
# O modelo retorna uma saída, que é uma previsão bruta (logits) para cada classe.
outputs = model(input_ids, attention_mask)

# Aplica a função softmax nos logits para converter os valores brutos em probabilidades de classe
# A função `softmax` garante que a soma das probabilidades seja 1 para cada exemplo no batch
# `dim=1` indica que a operação softmax deve ser aplicada ao longo da segunda dimensão (as colunas), ou seja, sobre as classes
# Isso faz com que as previsões se tornem probabilidades, onde a classe com a maior probabilidade será a prevista
predictions = F.softmax(outputs, dim=1)

print("Outputs: ", outputs)
print("Predictions: ", predictions)

## Treino

Procedimento de Treinamento e Otimização:

- Para replicar a metodologia de treinamento do artigo original do BERT, o processo de ajuste fino será conduzido utilizando o otimizador AdamW, fornecido pela biblioteca Hugging Face. Este otimizador é uma variante do Adam que implementa a correção do weight decay (decaimento de peso), alinhando-se com a abordagem descrita pelos autores para evitar o overfitting.

- Adicionalmente, será empregado um agendador linear de taxa de aprendizado (get_linear_schedule_with_warmup), configurado sem etapas de warm-up. Este agendador ajusta a taxa de aprendizado de forma gradual ao longo das épocas, contribuindo para uma convergência mais estável e eficiente do modelo.

In [ ]:
# --- Tratamento de Classes Desbalanceadas ---

# Importa a função específica para calcular os pesos das classes da biblioteca Scikit-learn.
from sklearn.utils.class_weight import compute_class_weight

# 1. Calcula os pesos para cada classe.
# Esta função ajuda a criar pesos que serão usados para penalizar mais os erros
# do modelo em classes com poucos exemplos durante o treinamento.
class_weights = compute_class_weight(
    # Parâmetro 'class_weight':
    # Define a estratégia para calcular os pesos.
    # 'balanced' -> Calcula os pesos de forma inversamente proporcional à frequência da classe.
    #               Ou seja, classes com menos amostras (raras) recebem um peso maior,
    #               e classes com muitas amostras (comuns) recebem um peso menor.
    class_weight="balanced",
    # Parâmetro 'classes':
    # Um array com todas as classes únicas presentes no seu dataset.
    # Ex: [0, 1, 2] ou ['negativo', 'neutro', 'positivo'].
    # np.unique() garante que pegamos cada classe apenas uma vez.
    classes=np.unique(df_train[sentiment_column]),
    # Parâmetro 'y':
    # Um array contendo os rótulos (targets) de todos os seus dados de treino.
    # A função usa este array para contar a frequência de cada classe
    # e aplicar a estratégia 'balanced'.
    y=df_train[sentiment_column].to_numpy(),
)

print(class_weights)

# 2. Converte os pesos (que estão em um array numpy) para um tensor do PyTorch.
# A função de perda do PyTorch (nn.CrossEntropyLoss) espera receber os pesos neste formato.
# -> 'dtype=torch.float': Garante que os pesos sejam números de ponto flutuante (ex: 32-bit float).
# -> '.to(device)': Move o tensor de pesos para o dispositivo de processamento correto (CPU ou GPU),
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

In [ ]:
# Função de perda (loss function)
# - CrossEntropyLoss é usada em tarefas de classificação multiclasse.
# - Ele compara a previsão do modelo (outputs) com a resposta correta (targets).
# - Calcula um único número (loss), que significa o quão "errada" foi a previsão.
# - Quanto maior a perda (loss), pior foi o desempenho do modelo no batch.
# - OBS - Caso o dataset seja desbalanceado, pode-se incluir pesos para cada classe.

if CLASS_WEIGHTS:
    # Erros em classes raras agora terão uma penalidade maior, forçando o modelo a aprendê-las.
    print("Usando pesos para classes desbalanceadas.")
    loss_fn = nn.CrossEntropyLoss(weight=weights).to(device)
else:
    print("Não usando pesos para as classes.")
    loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
# Otimizador AdamW
# - AdamW é o otimizador recomendado para Transformers.
# - Ele ajusta os pesos da rede neural durante o treinamento.
# - Com base no valor da perda calculado pela (loss_fn), ele determina como ajustar cada um dos milhões de pesos
# dentro do modelo, para que, da próxima vez, a perda seja menor. Ele implementa o algoritmo que efetivamente faz
# o modelo aprender.

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# optimizer = AdamW(
#     model.parameters(),
#     lr=LEARNING_RATE,
# )

# Número total de passos (batches) durante todo o treinamento
# Fórmula: total de lotes por época * número de épocas
total_steps = len(train_data_loader) * EPOCHS

# Passos de "aquecimento" (warmup) da taxa de aprendizado
# Durante ~10% dos primeiros passos, a learning rate cresce gradualmente
# Isso evita instabilidade no início do treino.
# num_warmup_steps = 0
num_warmup_steps = int(0.1 * total_steps)

# Scheduler (agendador) para a taxa de aprendizado
# - Ele controla um hiperparametro muito importante: a taxa de aprendizado (learning rate).
# Em vez de manter essa taxa fixa, o scheduler a ajusta dinamicamente durante o treinamento
# (aumentando no inicio e depois diminuindo), o que ajuda o modelo a convergir de forma mais rápida e estável.
# - Começa aumentando gradualmente a learning rate (warmup)
# - Depois decai linearmente até o final do treino
# Esse ajuste dinâmico ajuda o modelo a convergir melhor.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,  # número de passos para "aquecer"
    num_training_steps=total_steps,  # número total de passos de treino
)

Função auxiliar para treinar nosso modelo para uma época:

In [ ]:
from tqdm import tqdm  # Importa a biblioteca para criar barras de progresso visuais


def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    """
    Executa uma época de treinamento do modelo.

    Args:
        model: O modelo a ser treinado.
        data_loader: O DataLoader com os dados de treinamento.
        loss_fn: A função de perda.
        optimizer: O otimizador.
        device: O dispositivo (GPU/CPU) para treinamento.
        scheduler: O agendador de taxa de aprendizado.
        n_examples: O número total de exemplos de treinamento.

    Returns:
        Uma tupla contendo a acurácia e a perda média da época.
    """
    # Colocamos nosso modelo em "modo estudo".
    # Isso ativa camadas como o Dropout, que ajudam a evitar que o modelo "decore" as respostas.
    # É como dizer ao aluno: "Preste atenção, é hora de aprender, não de fazer prova."
    model = model.train()

    # Criamos "cadernos" para anotar o desempenho do modelo durante a aula.
    # 'losses' vai guardar a "nota" de erro de cada exercício.
    # 'correct_predictions' vai contar quantas respostas o aluno acertou.
    losses = []
    correct_predictions = 0

    total_steps = len(data_loader)  # Total de batches (Lotes de dados) no DataLoader
    print(f"Total de passos - batches: {total_steps}")
    # Usando tqdm para a barra de progresso com uma descrição clara
    progress_bar = tqdm(
        enumerate(data_loader, 1),
        total=total_steps,
        desc="Iniciando Treinamento",
        ncols=100,
    )

    # Para cada batch (lote de dados) de dados no DataLoader
    for batch in data_loader:

        # Move os 'input_ids' e 'attention_mask' para o dispositivo correto (GPU ou CPU)
        input_ids = batch["input_ids"].to(device)  # IDs dos tokens de entrada
        attention_mask = batch["attention_mask"].to(
            device
        )  # Máscara de atenção para ignorar tokens de padding
        targets = batch["targets"].to(device)  # Rótulos (sentimentos)

        # Passa os dados de entrada para o modelo (input_ids e attention_mask).
        # O modelo processa cada entrada. Se um batch tiver 16 entradas, processará 16 entradas.
        # outputs é a lista de saídas do modelo, que contém os logits para cada classe.
        # os logits são os valores brutos de previsão para cada classe. Ainda não são probabilidades.
        outputs = model(
            input_ids=input_ids,  # IDs dos tokens de entrada
            attention_mask=attention_mask,  # Máscara de atenção para ignorar tokens de padding
        )

        # Exemplo de saída (logits) para um batch com 2 exemplos e 5 classes:
        # tensor([
        #   [-0.0792,  0.1209,  0.0140,  0.1531,  0.1964],
        #   [-0.1822, -0.0655, -0.0424,  0.0147,  0.2308]
        # ])
        # Cada linha representa um exemplo no batch e cada coluna, um logit (pontuação) para uma classe.

        # Calcula a perda para o lote atual
        # Exemplo: se targets = tensor([4, 3]) e preds tensor([4, 4]),
        # a loss pode ser algo como: tensor(0.47)
        loss = loss_fn(outputs, targets)

        # Backpropagation
        # Aqui, descobrimos EXATAMENTE onde o modelo errou em seu raciocínio.
        # O PyTorch calcula um "gradiente" para cada peso do modelo, um mapa que aponta
        # a direção e a intensidade da correção necessária para cada "neurônio".
        loss.backward()

        # Aplica clipping nos gradientes para evitar explosões (exploding gradients)
        # Antes de corrigir, verificamos se o "mapa da correção" não é exagerado.
        # Se a correção for grande demais (explosão de gradiente), ela pode desestabilizar o modelo.
        # Essa linha garante que a correção seja firme, mas não destrutiva, no máximo 1.0
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # "A Correção de Fato"
        # O otimizador pega o mapa da correção e ajusta os pesos (o conhecimento) do modelo.
        # É neste exato momento que o aprendizado acontece!
        # Atualiza os pesos do modelo com base nos gradientes calculados.
        # Ele dá um pequeno passo na direção que deve diminuir a perda.
        optimizer.step()

        # "Virando a Página" (Zerando os Gradientes):
        # Após a correção, limpamos o "mapa" (os gradientes).
        # Se não fizéssemos isso, a correção do próximo batch seria somada com a do anterior,
        # criando uma bagunça e impedindo o aprendizado correto.
        optimizer.zero_grad()

        # "Ajustando a Estratégia de Ensino" (Passo do Scheduler):
        # O scheduler ajusta a "intensidade" da correção (taxa de aprendizado) para o próximo batch.
        # Atualiza a taxa de aprendizado usando o scheduler, seguindo seu plano de aquecimento e decaimento.
        scheduler.step()

        # --- Anotando o Desempenho (Métricas - não afetam o treinamento) ---

        # Obtém as classes previstas com maior valor de logit para cada exemplo
        # Exemplo de saída: tensor([4, 4])  -> significa que o modelo previu a classe 4 para ambos
        _, preds = torch.max(outputs, dim=1)

        # Soma o número de acertos: compara a previsão com o rótulo real
        # Exemplo: preds = tensor([4, 4]), targets = tensor([4, 3]) → 1 acerto
        correct_predictions += torch.sum(preds == targets)

        # Adiciona a perda atual à lista de perdas
        losses.append(loss.item())

        # Atualiza a barra de progresso com a perda média do lote
        progress_bar.set_postfix({"loss": np.mean(losses)})
        progress_bar.update()

    # Após todos os batches, calcula a acurácia e a média da perda
    accuracy = correct_predictions.double() / n_examples
    mean_loss = np.mean(losses)

    return accuracy, mean_loss

O scheduler (agendador) é chamado sempre que um batch (lote) é alimentado no modelo. Evitamos a explosão de gradientes cortando os gradientes do modelo usando [clip_grad_norm_](https://pytorch.org/docs/stable/nn.html#clip-grad-norm).

Vamos escrever outro que nos ajude a avaliar o modelo em um determinado carregador de dados:

In [ ]:
def eval_epoch(model, data_loader, loss_fn, device, n_examples):
    """
    Executa uma época de avaliação do modelo.

    Args:
        model: O modelo a ser avaliado.
        data_loader: O DataLoader com os dados de validação ou teste.
        loss_fn: A função de perda.
        device: O dispositivo (GPU/CPU) para avaliação.
        n_examples: O número total de exemplos de avaliação.

    Returns:
        Uma tupla contendo a acurácia e a perda média da época de avaliação.
    """
    model = model.eval()

    # Lista para armazenar as perdas e uma variável para contar as previsões corretas
    losses = []
    correct_predictions = 0

    # Desativa o cálculo de gradientes para a avaliação, economizando memória
    # e tempo de processamento.
    with torch.no_grad():
        # Para cada batch (lote de dados) de dados no DataLoader
        for d in data_loader:
            # Move os 'input_ids' e 'attention_mask' para o dispositivo correto (GPU ou CPU)
            input_ids = d["input_ids"].to(device)  # IDs dos tokens de entrada
            attention_mask = d["attention_mask"].to(
                device
            )  # Máscara de atenção para ignorar tokens de padding
            targets = d["targets"].to(device)  # Rótulos (sentimentos)

            # Passa os dados de entrada para o modelo (input_ids e attention_mask)
            # para obter as previsões
            # O modelo retorna uma saída, que é uma previsão bruta (logits) para cada classe.
            outputs = model(
                input_ids=input_ids,  # IDs dos tokens de entrada
                attention_mask=attention_mask,  # Máscara de atenção para ignorar tokens de padding
            )

            # Obtém as classes previstas com maior valor de logit (probabilidade bruta)
            _, preds = torch.max(outputs, dim=1)

            # Calcula a perda para o lote atual e a adiciona à lista
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # Soma o número de previsões corretas
            correct_predictions += torch.sum(preds == targets)

    # Calcula a acurácia (número de previsões corretas dividido pelo total de exemplos)
    accuracy = correct_predictions.double() / n_examples
    mean_loss = np.mean(losses)

    # Retorna a acurácia e a perda média
    return accuracy, mean_loss

Usando esses dois, podemos escrever nosso ciclo de treinamento. Também armazenaremos o histórico de treinamento.

Observe que estamos armazenando o estado do melhor modelo, indicado pelo maior validation accuracy (precisão de validação).

In [ ]:
%%time

# Dicionário para armazenar o histórico do treinamento (acurácia e perda)
history = defaultdict(list)
# Variável para armazenar a melhor acurácia de validação encontrada até o momento
best_accuracy = 0

# Loop principal que executa para cada época
for epoch in range(EPOCHS):

  print(f"Época {epoch + 1}/{EPOCHS}")
  print("-" * 10)

  # ====================
  # FASE DE TREINAMENTO
  # ====================
  # Chama a função para treinar o modelo por uma época e obtém a acurácia e a perda de treino

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train) # Passa o número total de exemplos de treinamento
  )

  print(f"Perda de Treino: {train_loss:.4f} | Acurácia de Treino: {train_acc:.4f}")

  # ====================
  # FASE DE VALIDAÇÃO
  # ====================
  # Chama a função para avaliar o modelo com os dados de validação
  val_acc, val_loss = eval_epoch(
      model,
      val_data_loader,
      loss_fn,
      device,
      len(df_val) # Passa o número total de exemplos de validação
  )

  print(f"Perda de Validação: {val_loss:.4f} | Acurácia de Validação: {val_acc:.4f}")
  print("\n")

  # ====================
  # ARMAZENA O HISTÓRICO DE TREINAMENTO
  # ====================
  # Armazena a acurácia e a perda da época no histórico
  history['train_acc'].append(train_acc)  # Adiciona a precisão de treinamento ao histórico
  history['train_loss'].append(train_loss)  # Adiciona a perda de treinamento ao histórico
  history['val_acc'].append(val_acc)  # Adiciona a precisão de validação ao histórico
  history['val_loss'].append(val_loss)  # Adiciona a perda de validação ao histórico

  # ====================
  # SALVA O MELHOR MODELO (melhor acurácia de validação)
  # ====================
  # Esta é uma boa prática para evitar o overfitting e garantir
  # que o melhor modelo seja utilizado.
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Visualização do Histórico de Treinamento

In [ ]:
# ==============================================================================
# VISUALIZAÇÃO DO HISTÓRICO DE TREINAMENTO
# ==============================================================================

# ------------------------------------------------------------------------------
# GRÁFICO 1: ACURÁCIA
# ------------------------------------------------------------------------------

train_acc = [x.cpu().item() if hasattr(x, "cpu") else x for x in history["train_acc"]]
val_acc = [x.cpu().item() if hasattr(x, "cpu") else x for x in history["val_acc"]]

# Cria um gráfico para visualizar a acurácia de treinamento e validação ao longo das épocas
plt.plot(train_acc, label="acurácia de treino")
plt.plot(val_acc, label="acurácia de validação")
plt.title("Acurácia de Treino vs. Validação")
plt.ylabel("Acurácia")
plt.xlabel("Épocas")
plt.legend()
plt.ylim([0, 1])
plt.show()

# ------------------------------------------------------------------------------
# GRÁFICO 2: PERDA (LOSS)
# ------------------------------------------------------------------------------

# Supondo que seu dicionário 'history' contenha 'train_loss' e 'val_loss'
train_loss = [x.cpu().item() if hasattr(x, "cpu") else x for x in history["train_loss"]]
val_loss = [x.cpu().item() if hasattr(x, "cpu") else x for x in history["val_loss"]]

# Cria um gráfico para visualizar a acurácia de treinamento e validação ao longo das épocas
plt.plot(train_loss, label="perda de treino")
plt.plot(val_loss, label="perda de validação")
plt.title("Perda de Treino vs. Validação")
plt.ylabel("Perda")
plt.xlabel("Épocas")
plt.legend()
plt.show()

## AVALIAÇÃO FINAL DADOS DE TESTE

#### Obter um modelo já treinado da internet

In [ ]:
# !gdown 16Xq3bNEx7Owf7kOMjfgQu-BkWsX7hEpV -O best_model_state.bin

# model = SentimentClassifier(len(class_names))
# model.load_state_dict(torch.load('best_model_state.bin'))
# model = model.to(device)

#### Carregar o modelo salvo localmente

In [ ]:
# ==============================================================================
# AVALIAÇÃO FINAL DO MELHOR MODELO
# ==============================================================================

# Definindo o caminho para o modelo salvo
model_path = os.path.join(os.getcwd(), "best_model_state.bin")

# Carregar o modelo
model = SentimentClassifier(
    len(class_names)
)  # Inicialize o modelo com a mesma arquitetura
model.load_state_dict(torch.load(model_path))  # Carregar o estado do modelo salvo
model = model.to(device)  # Enviar o modelo para o dispositivo (GPU ou CPU)

#### Avaliação

In [ ]:
# Vamos começar calculando a precisão dos dados de teste:

test_acc, _ = eval_epoch(model, test_data_loader, loss_fn, device, len(df_test))

test_acc.item()

Função para fazer previsões em um conjunto de dados.


Semelhante à função de avaliação, exceto que armazenamos o texto das revisões e as probabilidades previstas (aplicando o softmax nos resultados do modelo)

In [ ]:
def get_predictions(model, data_loader):
    # Define o modelo no modo de avaliação, desativando camadas como dropout e batchnorm
    model = model.eval()

    # Listas para armazenar os resultados
    review_texts = []  # Textos dos reviews
    predictions = []  # Classes previstas pelo modelo
    prediction_probs = []  # Probabilidades associadas às previsões
    real_values = []  # Valores reais (rótulos verdadeiros

    # Desativa o cálculo do gradiente, economizando memória e acelerando o processo
    with torch.no_grad():
        # Para cada batch (lote de dados) de dados no DataLoader
        for batch in data_loader:

            texts = batch["review_text"]  # Obtém os textos dos reviews

            # Move os 'input_ids' e 'attention_mask' para o dispositivo correto (GPU ou CPU)
            input_ids = batch["input_ids"].to(device)  # IDs dos tokens de entrada
            attention_mask = batch["attention_mask"].to(
                device
            )  # Máscara de atenção para ignorar tokens de padding
            targets = batch["targets"].to(device)  # Rótulos (sentimentos)

            # Passa os dados pelo modelo para obter os logits (saídas brutas)
            # model retorna os logits, que são as previsões brutas para cada classe
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Obtém a classe prevista que possui o maior valor de logit (probabilidade bruta)
            _, preds = torch.max(outputs, dim=1)

            # Aplica a função softmax nos logits para converter os valores brutos em probabilidades de classe
            # A função `softmax` garante que a soma das probabilidades seja 1 para cada exemplo no batch
            # `dim=1` indica que a operação softmax deve ser aplicada ao longo da segunda dimensão (as colunas), ou seja, sobre as classes
            # Isso faz com que as previsões se tornem probabilidades, onde a classe com a maior probabilidade será a prevista
            probs = F.softmax(outputs, dim=1)

            # Armazena os resultados
            review_texts.extend(texts)  # Adiciona os textos dos reviews
            predictions.extend(preds)  # Adiciona as classes previstas
            prediction_probs.extend(probs)  # Adiciona as probabilidades das previsões
            real_values.extend(targets)  # Adiciona os rótulos verdadeiros

    # Converte listas de tensores em um único tensor e move para a CPU
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

Obtendo as previsões do modelo no conjunto de teste

In [ ]:
# y_review_texts: Lista contendo os textos dos reviews do conjunto de teste.
# y_pred: Tensor com as classes previstas pelo modelo.
# y_pred_probs: Tensor com as probabilidades associadas a cada classe.
# y_test: Tensor com os rótulos verdadeiros.

# Obtendo as previsões do modelo no conjunto de teste
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
    model,  # Modelo treinado para análise de sentimentos (SentimentClassifier)
    test_data_loader,  # DataLoader com os dados de teste, incluindo textos, input_ids e targets
)

Imprime o relatório de classificação:

In [ ]:
# y_test: vetor contendo os rótulos verdadeiros.
# y_pred: vetor contendo as classes previstas pelo modelo.


print("\n" + "=" * 50)
print("Relatório de Classificação no Conjunto de Teste")
print("=" * 50 + "\n")
print(classification_report(y_test, y_pred, target_names=class_names))

Matriz de confusão:

In [ ]:
# ==============================================================================
# ANÁLISE DA MATRIZ DE CONFUSÃO
# ==============================================================================


# Cria a matriz de confusão
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha="right")
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha="right")
    plt.ylabel("Verdadeiro Sentimento")
    plt.xlabel("Sentimento Previsto")


cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
print("\n" + "=" * 50)
print("Matriz de Confusão no Conjunto de Teste")
print("=" * 50 + "\n")
show_confusion_matrix(df_cm)
plt.show()

### ANÁLISE DE UM EXEMPLO DO CONJUNTO DE TESTE

In [ ]:
# ==============================================================================
# ANÁLISE DE UM EXEMPLO DO CONJUNTO DE TESTE
# ==============================================================================

# Seleciona o índice do review que deseja analisar
idx = 10

review_text = y_review_texts[idx]  # Obtém o texto do review no índice especificado

# Obtém o rótulo verdadeiro (sentimento real) para esse review
true_sentiment = y_test[idx].item()  # .item() para converter de tensor para inteiro

# Cria um DataFrame com as classes e as probabilidades previstas pelo modelo para esse review
pred_df = pd.DataFrame(
    {
        "class_names": class_names,  # Lista com os nomes das classes
        "values": y_pred_probs[
            idx
        ].numpy(),  # Converte o tensor de probabilidades para NumPy array
    }
)

# Exibe o texto do review, o sentimento real e as probabilidades previstas
print(f"Review: {review_text}")
print(
    f"Sentimento Real: {class_names[true_sentiment]}"
)  # Mostra o nome da classe em vez do índice
print(pred_df)


# Cria o gráfico de barras com as probabilidades
sns.barplot(x="values", y="class_names", data=pred_df, orient="h")
plt.ylabel("Sentimento")
plt.xlabel("Probabilidade")
plt.xlim([0, 1])
plt.show()

### PREVISÃO EM TEXTO BRUTO

In [ ]:
review_text = "O aplicativo IDR-Peixe muito bom, recomendo a todos."

encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    padding="max_length",  # Changed from pad_to_max_length=True
    return_attention_mask=True,
    return_tensors="pt",
)

Vamos obter as previsões do nosso modelo:

In [ ]:
input_ids = encoded_review["input_ids"].to(device)
attention_mask = encoded_review["attention_mask"].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f"Review text: {review_text}")
print(f"Sentiment  : {class_names[prediction]}")